# How do we prepare data for a bike share analytics platform?

## Goals

In this case, we will introduce an unclean dataset - the type of dataset that you will most likely be working with in a job setting. We hope to provide you with a framework for tackling some common problems of unclean datasets (missing values, typos, etc.). You will learn to assess the cleaning needs of a dataset and come up with strategies to prepare it appropriately for further analysis.

## Introduction

**Business Context.** You are a data science consultant for a London-based bike share company. The company has been collecting data about trips taken on each of the public transportation system bikes (Santander Cycles) to detect demand patterns that could be used to refine their own business model. Since the dataset collected is quite large and increasing by the day, they have subscribed to a new analytics platform which gives them information and insights as soon as trip data is fed into it. However, the analytics platform requires the collected data to be cleaned and converted into a certain format, for which the client requires your help.

**Business Problem.** Your task is to **transform the raw data the client has into a format that can be fed into their analytics platform, and add additional features that the client would find useful to have in their platform.**

**Analytical Context.** The data provided by the client is spread across 2 files. The first file is a CSV file that contains trip data from 2020 and 2021 (`trips.csv`), with features such as trip time, start and end stations, and bike number. The second file is a JSON file containing details about the stations (`stations.json`).

Two of the features that your client considers should be added to the dataset are journey IDs and total cost per rental, but you have freedom to come up with more features if you deem it useful. In London, [when you hire a Santander bike](https://www.visitlondon.com/traveller-information/getting-around-london/london-cycle-hire-scheme), you pay £2 upfront for the right to use any bike in the city within 24 hours, for up to 30 minutes. However, if you exceed this threshold, you are charged an additional £2 for each additional 30 minutes.

## A framework for data preparation

There's no universally agreed-upon method to follow to clean datasets, but we propose to you the following, which divides the process into four components. Most sources will coincide with this in some form:

![Components of data cleaning](data/images/components_data_cleaning.jpg)

Let's break this down, relating each component to the business problem at hand.

### Format

![Format](data/images/format.jpg)

You always need to make sure you deliver your cleaned data using the right **file formats** and **encoding** so that it can be consumed by the end user. Some popular file formats include:

* `.csv`: Comma-Separated Value files. This is your standard rows-and-columns table format that you've seen in DataFrames. A common way of sharing `.csv` files is compressing them as `.csv.gz` files, which greatly reduces their size. You can [do that](https://stackoverflow.com/a/46073397/6945498) in `pandas` pretty easily.
* `.json`: JSON files. These are very similar to Python dictionaries. They are popular alternatives to `.csv` files because they usually consume a lot less memory, but they are also less human-readable. We will cover these later in this case.
* `.xlsx`: Excel files. Generally, you should avoid using the deprecated Excel `.xls` format ([here's](https://www.bbc.com/news/technology-54423988) a reason why).
* `.sql`: SQL files. These can be directly loaded into a SQL database. These are covered in a different case.

There are other formats that `pandas` is capable of exporting to (check the [docs](http://pandas-docs.github.io/pandas-docs-travis/user_guide/io.html) for more information). In our case, the bike share company didn't ask for a specific format, so giving the final output as a `.csv` file should be a safe option.

Now that we've decided on the format, let's think about the encoding. Encodings are basically glossaries that translate between binary numbers (numbers made up of just zeros and ones - which is what your computer understands) and human-language characters. There are quite a few encodings, but the international standard nowadays is Unicode, specifically [`UTF-8`](https://en.wikipedia.org/wiki/UTF-8), and should be your first choice unless you are explicitly required to use a different one.

### Consistency

![Consistency](data/images/consistency.jpg)

When we talk about achieving consistency in a dataset, we are referring to the following:

1. **Data type consistency.** This refers to having all the features represented in the right data type. Thus, if you're working in `pandas`, decimal numbers must be represented as `float`, integers as `int`, strings as `object`, categories as `category`, and so on. Other examples of data type consistency are converting Yes/No columns into booleans (`True`, `False`), making sure all dates are correctly formatted and are not treated as strings or numbers, and rounding decimal numbers to an appropriate precision.
    1. As an aside, keep in mind that `pandas` doesn't allow missing values in columns of the `int` data type, so if you need to have `NaN`s in your Series, you'll need to use `float`.
2. **Unit consistency.** If you have numerical features, it is important that they are in the right units. For example, if you have one feature in kilometers and another one in miles, you have to convert them to a common unit before you can do math involving both of them.
3. **Categorical consistency.** Features that are supposed to be categorical should be standardized. For instance, if your dataset has one category `bike` that is represented by two different strings (e.g. `bike` and `bicycle`), then you have to detect this inconsistency and correct it (e.g. by replacing all instances of `bicycle` with `bike`, to have only one label per category).
4. **String formatting.** Sometimes you will have extraneous elements in your strings, like asterisks and other special characters. These are often not useful, so you need to remove them. Other string formatting tasks might include removing punctuation, capitalizing, lowercasing or uppercasing, removing leading and trailing spaces, etc.
5. **Referential integrity.** Each unique observation in your dataset should have a unique ID, especially if you plan to join it with other datasets (in which case the IDs of your observations must match the join keys of the other dataset). Often times, this requires that you remove duplicate rows and create IDs for rows that don't have them.
6. **Column name consistency.** You don't have to be as strict with this one as with the previous requirements, but being consistent with how you name columns is nonetheless a good practice. You can set all your columns to follow a common naming standard so that their names are meaningful and referring to them in the future is easy.

### Exercise 1

Run the cell below and explore the report that appears. It summarizes the `trips.csv` dataset. Then write your own assessment of the data cleaning needs of this dataset *for each* of the "Consistency" requirements outlined above. Here are the first 10 rows:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Rental Id</th>      <th>Duration</th>      <th>Bike Id</th>      <th>End Date</th>      <th>EndStation Id</th>      <th>Start Date</th>      <th>StartStation Id</th>      <th>tag</th>      <th>userCategory</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>101428476</td>      <td>1680</td>      <td>12829.0</td>      <td>03/09/2020 20:32</td>      <td>132.0</td>      <td>03/09/2020 20:04</td>      <td>574</td>      <td>Priority low</td>      <td>['A']</td>    </tr>    <tr>      <th>1</th>      <td>101522714</td>      <td>2700</td>      <td>10863.0</td>      <td>06/09/2020 12:56</td>      <td>702.0</td>      <td>06/09/2020 12:11</td>      <td>82</td>      <td>Priority low</td>      <td>['A']</td>    </tr>    <tr>      <th>2</th>      <td>101377356</td>      <td>420</td>      <td>3997.0</td>      <td>02/09/2020 10:56</td>      <td>97.0</td>      <td>02/09/2020 10:49</td>      <td>225</td>      <td>priority low</td>      <td>['A']</td>    </tr>    <tr>      <th>3</th>      <td>101393663</td>      <td>660</td>      <td>16542.0</td>      <td>02/09/2020 18:40</td>      <td>622.0</td>      <td>02/09/2020 18:29</td>      <td>97</td>      <td>priority low</td>      <td>['A']</td>    </tr>    <tr>      <th>4</th>      <td>101622659</td>      <td>660</td>      <td>1605.0</td>      <td>08/09/2020 19:57</td>      <td>219.0</td>      <td>08/09/2020 19:46</td>      <td>36</td>      <td>priority_high</td>      <td>['A']</td>    </tr>    <tr>      <th>5</th>      <td>101622643</td>      <td>600</td>      <td>9046.0</td>      <td>08/09/2020 19:55</td>      <td>37.0</td>      <td>08/09/2020 19:45</td>      <td>36</td>      <td>Priority low</td>      <td>['B']</td>    </tr>    <tr>      <th>6</th>      <td>101538934</td>      <td>1740</td>      <td>17447.0</td>      <td>06/09/2020 16:56</td>      <td>709.0</td>      <td>06/09/2020 16:27</td>      <td>158</td>      <td>priority_medium</td>      <td>['A']</td>    </tr>    <tr>      <th>7</th>      <td>101538943</td>      <td>1860</td>      <td>9294.0</td>      <td>06/09/2020 16:58</td>      <td>708.0</td>      <td>06/09/2020 16:27</td>      <td>158</td>      <td>priority_high</td>      <td>['A']</td>    </tr>    <tr>      <th>8</th>      <td>101411350</td>      <td>660</td>      <td>2011.0</td>      <td>03/09/2020 12:05</td>      <td>235.0</td>      <td>03/09/2020 11:54</td>      <td>558</td>      <td>priority_high</td>      <td>['A']</td>    </tr>    <tr>      <th>9</th>      <td>101605095</td>      <td>1440</td>      <td>18207.0</td>      <td>08/09/2020 14:58</td>      <td>19.0</td>      <td>08/09/2020 14:34</td>      <td>281</td>      <td>Priority low</td>      <td>['A']</td>    </tr>  </tbody></table>

**Hint:** The list of available features and their data types is as follows (please notice that the `object` data type in the report is called "Categorical", although it is *not* the same as the `pandas` `category` data type):

1. **Rental Id:** A user can purchase the right to pick any bike in the city within 24 hours for £2. This column contains the ID of the rental (which may correspond to more than one bike). Notice that this is *not* the ID of the journey. Data type: `int64`.
1. **Duration:** The duration of the journey in seconds. Data type: `int64`.
2. **Bike Id:** The ID of the bike. Data type: `int64`.
3. **End Date:** End time of the journey. Data type: `object`.
4. **EndStation Id:** ID of the station at which this journey ended. Data type: `int64`.
5. **Start Date:** Start time of the journey. Data type: `object`.
6. **StartStation Id:** ID of the station at which this journey started. Data type: `int64`.
7. **tag:** A tag that one of the members of your team assigned to each journey to make it easier to group them for further analysis. This column was not part of the original dataset. Data type: `object`.
8. **userCategory:** Can be either `A` (occasional user) or `B` (frequent user). Data type: `object`.

For the purposes of this exercise, you don't need to worry about the "Reproduction" tab in the report below. Do make sure to scroll down within the below cell and inspect the entire report though.

In [1]:
from IPython.display import IFrame
IFrame(src='data/trips_report.html', width=950, height=700)

**Answer.**

End station IDs should be integers
Bike IDs should be integers
Tag is inconsistent
Date needs to be in date, time format
user category doesn't need to be an array

-------

### Relevance

![Relevance](data/images/relevance.jpg)

Guaranteeing that your dataset contains only the relevant information implies:

1. **Removing unnecessary features or observations.** This depends on what your client would like to use the data for. Unnecessary columns could be columns that contain the same information as other columns, only in a different data type, or columns that simply do not add interesting information. For example, unnecessary observations can be for years we aren't interested in (e.g. "drop everything before August 2013") or categories that are not needed for the use case (e.g. "drop non-registered users").
2. **Staying at the right level of granularity.** If your client needs the data aggregated by month, then deliver it aggregated by month. If they need it aggregated by state instead of zip code, then do that. In general though, unless required otherwise, the recommendation is to always provide the highest level of granularity, that is, the most detailed level of resolution of the data that you can, because while you can easily convert disaggregated data into aggregated data (e.g. taking daily data and grouping by month), it is impossible to do the reverse (convert monthly data to daily data).
3. **Properly dealing with missing values.** There are three basic strategies:
    1. Drop the rows with missing values
    2. Come up with reasonable estimates of what the values should be were they available (this is called **interpolation**)
    3. Replace them with an appropriate placeholder (`Not available` is a common one for string variables, and the numbers 0 or -1 are sometimes used for numeric variables)
4. **Detecting outliers and dealing with them.** Properly handling outliers is more important at the data modeling stage than the data cleaning stage. Even so, when cleaning datasets you are expected to remove (and possibly replace) outliers that are obviously erroneous data. For example, while in the data cleaning stage you should remove a person who traveled 120 kilometers in 10 minutes on their bike (clearly erroneous data), but if they instead traveled that same distance in one day instead, you should leave it as is because even though the observation is still an outlier, it might still be perfectly valid.

### Exercise 2

Assess the cleaning needs of this dataset for *each* of the "Relevance" requirements above.

**Answer.**

-------

### Exercise 3

`Bike Id` is one of the columns with missing values. Which of the three strategies explained earlier do you think is most appropriate?

**Answer.**

-------

### Exercise 4

The next column we can concentrate on is `userCategory`. We know that it can have only one of 2 values: `A` or `B`.

Suppose that one thing (but not the only thing) our client cares about is a rough idea of how the number of riders per user category has changed over time. Which of the missing value strategies would be most appropriate?

**Answer.**

-------

### Exercise 5

Come up with a proper method for dealing with missing values in `EndStation Id`.

**Answer.**

-------

## Data augmentation

![Data augmentation](data/images/data_augmentation.jpg)

Data augmentation usually involves two things - **merging** our dataset with other datasets, and **feature engineering**.

To merge datasets, we need to have compatible IDs for each one of them (which is a referential integrity requirement). In our case, we need to merge the `trips.csv` dataset with the `stations.json` dataset (here we only show the first 10 rows of the latter after we read in the JSON file and transform it into a table):

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Latitude</th>      <th>Longitude</th>      <th>Station Name</th>    </tr>    <tr>      <th>Station ID</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>1</th>      <td>51.529163</td>      <td>-0.109970</td>      <td>River Street , Clerkenwell</td>    </tr>    <tr>      <th>2</th>      <td>51.499606</td>      <td>-0.197574</td>      <td>Phillimore Gardens, Kensington</td>    </tr>    <tr>      <th>3</th>      <td>51.521283</td>      <td>-0.084605</td>      <td>Christopher Street, Liverpool Street</td>    </tr>    <tr>      <th>4</th>      <td>51.530059</td>      <td>-0.120973</td>      <td>St. Chad's Street, King's Cross</td>    </tr>    <tr>      <th>5</th>      <td>51.493130</td>      <td>-0.156876</td>      <td>Sedding Street, Sloane Square</td>    </tr>    <tr>      <th>6</th>      <td>51.518117</td>      <td>-0.144228</td>      <td>Broadcasting House, Marylebone</td>    </tr>    <tr>      <th>7</th>      <td>51.534300</td>      <td>-0.168074</td>      <td>Charlbert Street, St. John's Wood</td>    </tr>    <tr>      <th>8</th>      <td>51.528341</td>      <td>-0.170134</td>      <td>Lodge Road, St. John's Wood</td>    </tr>    <tr>      <th>9</th>      <td>51.507385</td>      <td>-0.096440</td>      <td>New Globe Walk, Bankside</td>    </tr>    <tr>      <th>10</th>      <td>51.505974</td>      <td>-0.092754</td>      <td>Park Street, Bankside</td>    </tr>  </tbody></table>

We got lucky with this dataset. Its ID (the `Station ID` column, in bold) is the same as the categories in `StartStation Id` and `EndStation Id`, so merging should be easy.

### Exercise 6

**Feature engineering** means that you create new features in the dataset using existing features. We've already mentioned a feature that we would like to create - the ID of each trip. What other features do you think could be interesting to our client?

**Answer.**

-------

Feature engineering is a very important task in any data workflow. A special case is feature engineering on textual datasets, which is very often used in Natural Language Processing (NLP). We won't cover the very specific methods and tools used in NLP here, but you will study them in future cases.

## Conclusion

We cleaned up some messy client data and added some new features based on their requests. We took a small sample of the overall data so we could work with it locally. However, cleaning up the data was not merely some rote mechanical work. We had to use our common sense and judgment of likely use cases of this data in order to determine how to fix it. Additionally, we had to consider the potential impact of our changes and whether it would adversely impact potential uses of the data in the future, even if those were not likely uses at this time.

Data wrangling & cleaning is arguably the most important step of the entire data science process, without which we could have wrong or corrupt results. In practice, data professionals can spend upwards of 60 - 70% of their time on cleaning and organizing their data.

## Takeaways

In this case, you learned a framework for cleaning datasets, which consists of four components:

1. Ensuring your output has the right encoding and is in the right format
2. Making your dataset internally and externally consistent
3. Keeping only the relevant information
4. Augmenting your data with new datasets and engineered features

As always, you have to keep in mind the needs of the users of your output, since not all data cleaning tasks are appropriate in all situations. Nonetheless, you can use this framework as a checklist to be sure that you do not leave anything important out.

When cleaning data, one very common problem is missing data. We learned a few ways of dealing with missing data and when these methods tend to be appropriate:

1. Removing the rows with missing data entirely is a nuclear option that only makes sense when you have a clear and very limited use case for the dataset.
2. Replacing the missing values with a meaningless filler like "Not found" or 0 makes sense if you need to preserve the other data in those rows but either cannot sensibly fill in or don't particularly care about the missing values in that specific column.
3. Interpolating the missing values makes sense when you need to preserve elements of the underlying distribution or trends within the data of that column.

We also saw that for large datasets, working with a small subset of the data is a useful tool for gaining intuition about the dataset and rapidly iterating on the cleaning process. This intuition gathering element cannot be emphasized enough - in fact, as you saw in Exercise 1, a big part of the cleaning process involved elements of exploratory data analysis, via generating summary statistics.

## Attribution

"Cycle Hire trip data" (modified from the original), 11 Feb, 2021, Transport for London, [Transport for London Licence](https://tfl.gov.uk/corporate/terms-and-conditions/transport-data-service), https://tfl.gov.uk/info-for/open-data-users/our-open-data.